# # setup
---

In [ ]:
%cd /data/kimgh/CenterPoint-static/det3d/ops/iou3d_nms

!python setup.py build_ext --inplace

# # Prepare data
---

In [ ]:
%cd /data/kimgh/CenterPoint

!python create_data.py nuscenes_data_prep --root_path '/data/kimgh/CenterPoint/data/nuScenes/' --version 'v1.0-mini' --nsweeps 10

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/data'
sensor ='lidar'

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor}

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/lidar'
sensor = 'lidar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/radar'
sensor = 'radar'
subsample = 10
num_process = 10

f'!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

root_path = '/data/kimgh/CenterPoint-custom/CenterPoint-static/data/etc/test'
sensor = 'lidar'
subsample = 10
num_process = 10

!python create_data.py nia_data_prep --root_path {root_path} --sensor {sensor} --subsample {subsample} --num_process {num_process}

# # Train
---
- pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_lidar" --no-autorestart -- 4 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py --work_dir ./result/lidar_1stsub/train --validate

- CUDA_VISIBLE_DEVICES=1,2 pm2 start train_multigpu.sh --interpreter bash --name "centerpoint_train_radar" --no-autorestart -- 2 --config /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_train.py --work_dir ./result/selectsub/train/radar --validate

In [ ]:
%cd /data/kimgh/CenterPoint

CONFIG_PATH = '/data/kimgh/CenterPoint/configs/nusc/voxelnet/nusc_centerpoint_voxelnet_01voxel.py'
work_dir = './result/nusc_centerpoint_voxelnet_01voxel'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = './result/lidar/train'

!python train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_train.py'
work_dir = './result/selectsub/train/lidar'

f'!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate'

In [ ]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-dynamic

CONFIG_PATH = '/data/kimgh/CenterPoint-custom/CenterPoint-dynamic/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel.py'
work_dir = './result/selectsub/train/lidar'

!python -m torch.distributed.launch --nproc_per_node 4 train.py --config {CONFIG_PATH} --work_dir {work_dir} --validate

# # Validation
---

In [42]:
# lidar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/test/lidar/normal_epoch24'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/lidar/epoch_24.pth'
sensor = 'lidar'

# f'!CUDA_VISIBLE_DEVICES=1,2,3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
!CUDA_VISIBLE_DEVICES=0,1 bash test_multigpu.sh 2 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test

/data/kimgh/CenterPoint-custom/CenterPoint-static
35145
/home/ubuntu/anaconda3/envs/nia48_310/lib/python3.10/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
No APEX!
No APEX!
no apex
no apex
2023-05-02 16:23:13.405468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized t

In [ ]:
# lidar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/test/lidar/abnormal_epoch22'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/lidar/epoch_22.pth'
sensor = 'lidar'

f'!CUDA_VISIBLE_DEVICES=1,2,3 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'
f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

In [ ]:
# radar-normal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_normal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/test/radar/normal_epoch25'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/radar/epoch_25.pth'
sensor = 'radar'

!CUDA_VISIBLE_DEVICES=1 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
# f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

In [ ]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py'
work_dir = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/test/radar/abnormal_epoch15'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/radar/epoch_15.pth'
sensor = 'radar'

# !CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

In [ ]:
# radar-abnormal

%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/radar/nia_centerpoint_voxelnet_01voxel_radar_abnormal.py'
work_dir = '/data/kimgh/test'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/radar/epoch_15.pth'
sensor = 'radar'

# !CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
!CUDA_VISIBLE_DEVICES=0,1 bash test_multigpu.sh 2 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test

In [38]:
%cd /data/kimgh/CenterPoint-custom/CenterPoint-static

config = '/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal_copy.py'
work_dir = '/data/kimgh/test'
checkpoint = '/data/kimgh/CenterPoint-custom/CenterPoint-static/result/selectsub/train/lidar/epoch_22.pth'
sensor = 'lidar'

!CUDA_VISIBLE_DEVICES=0 python dist_test.py --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test
# f'!CUDA_VISIBLE_DEVICES=1,2,3 bash test_multigpu.sh 3 --config {config} --work_dir {work_dir} --checkpoint {checkpoint} --sensor {sensor} --speed_test'

/data/kimgh/CenterPoint-custom/CenterPoint-static
No APEX!
no apex
2023-05-02 15:38:02.986274: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 15:38:04.093006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No Tensorflow
Deformable Convolution not built!
No APEX!
/data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal_copy.py
2023-05-02 15:38:04,876 - INFO - Sensor: lidar
2023-05-02 15:38:04,876 - INFO - Config: /data/kimgh/CenterPoint-custom/CenterPoint-static/configs/nia/lidar/nia_centerpoint_voxelnet_01voxel_lidar_normal_copy.py
2023-05-02 15:38:04,876 - INFO - Distributed testing: False
2023-05-02 15:38:04,876 - INFO - torch.backends.